In [1]:
import os
import functools
import itertools
import string

import pandas as pd
import numpy as np
from patsy import dmatrices
from scipy import stats
from estimagic import minimize, maximize
from estimagic.differentiation.derivatives import first_derivative
from respy.method_of_simulated_moments import _harmonize_input, get_flat_moments
import estimagic
from estimagic import maximize
from estimagic.inference.likelihood_inference import do_likelihood_inference

In [2]:
estimagic.__version__

'0.1.2'

# Tutorial Standart Errors MSM 
## Ordered Logit Example
This notebook contains a full tutorial about standart errors with method of simuated moments.
We continue the maximum likelihood example and still consider an ordered logit case.
- I particular we simulate a dataset with a data generating process given by a simple ordered logit model.

- We simulate data and only use discrete values of the independent variable because it makes the method of moments example more explicit.

- We take the simulated dataset and build conditional moments of the outcome variable for each combination of independent variables.

- Since these variables are discrete there is a finite number of such moments and  we do not need to discretize any data.

- Thereafter we discard the observed dependent variable and only keep the "observed" moments and observed independent variables.

- The objective function takes an arbitrary parameter vector and the observed 
  independent variables and maps them into implied population moments.
  Particularly it uses these inputs and generates outcomes for each individual   with the ordered logit model.

- Then it builds simulated moments with these outcomes and compares them to the  observed moments.

- The opimization procedure choses the parameter vector that minimizes the distance between observed and simulated moments.

- Then we build standart errors. Importantly we get weighting matrix and the asymptotic variance matrix with a bootstrap procedure.

- Sources for standart errors:
https://onlinelibrary.wiley.com/doi/abs/10.3982/ECTA7560
https://eml.berkeley.edu/choice2/ch10.pdf
(Please add any references)

- Altough this is not strictly necessary in our case it is generally required in  any example that uses higher order moments in the data.

- Thereafter it is important to correct for the amount of bootstrap samples.

- We compare standart errors to maximum likelihood and find that they are approximately similar. 


## Functions

### Generate a simple dataset with ordered logit dgp 

In [3]:
def simulate_dataset(n_agents, params):
    beta = params.loc["beta", "value"].to_numpy()
    cutoffs = params.loc["cutoff", "value"].to_numpy()
    range_vars = np.random.choice(range(2,4),size=len(beta))
    X = np.concatenate(
        [np.random.choice(range(x),size=n_agents).reshape(n_agents,1) for x in range_vars],axis=1)

    # calculate deterministic part of utilities
    xb = X.dot(beta).reshape(n_agents,1)

    # Simulate Result:
    upper_cutoffs = np.hstack([cutoffs, np.inf])
    lower_cutoffs = np.hstack([-np.inf, cutoffs])
    upper_cdf = stats.logistic.cdf(upper_cutoffs - xb)
    lower_cdf = stats.logistic.cdf(lower_cutoffs - xb)

    prob_cumulative = (upper_cdf - lower_cdf).cumsum(axis=1)
    draws = np.random.rand(len(xb), 1)
    labels = (draws < prob_cumulative).argmax(axis=1)
    out = pd.DataFrame(X)
    out.columns = params.loc["beta"].index.values
    out["y"] = labels
    return out

    
    

In [4]:
def _build_data_df(x,y,cols):
    # Basic utility
    data = np.concatenate([x,y.reshape(len(y),1)],axis=1)
    return pd.DataFrame(data=data.copy(),columns=cols)

### Processing similar to the Likelihood example

In [5]:
def ordered_logit_processing(formula, data):
    """Process user input for an ordered logit model."""
    # extract data arrays
    y, x = dmatrices(formula + " - 1", data, return_type="dataframe")
    y = y[y.columns[0]]

    # extract dimensions
    num_choices = len(y.unique())
    beta_names = list(x.columns)
    num_betas = len(beta_names)
    num_cutoffs = num_choices - 1

    # set-up index for params_df
    names = beta_names + list(range(num_cutoffs))
    categories = ["beta"] * num_betas + ["cutoff"] * num_cutoffs
    index = pd.MultiIndex.from_tuples(zip(categories, names), names=["type", "name"])

    # make params_df
    np.random.seed(5471)
    start_params = pd.DataFrame(index=index)
    start_params["value"] = np.hstack(
        [
            np.random.uniform(low=-0.5, high=0.5, size=len(x.columns)),
            np.arange(num_cutoffs) * 2,
        ]
    )
    start_params["group"] = start_params.index.get_level_values("type")

    # make constraints
    constr = [{"loc": "cutoff", "type": "increasing"}]

    # turn pandas objects into numpy arrays
    y_arr = y.to_numpy().astype(int)
    x_arr = x.to_numpy()
    
    return start_params, y_arr, x_arr, constr

In [6]:
def _build_moments(data, ind):
    """
    Map data into population moments.
    data: pd.DataFrame that contains independent and dependent
    variables for each individual
    ind: list of independent variables that are used for conditional moments
    """
    im = data.copy()
    #im["gpa"] = pd.qcut(im.gpa,q=3,labels=False)
    ranges = data.max(axis=0)
    ix = pd.MultiIndex.from_tuples(itertools.product(*(range(int(x + 1)) for x in ranges)))
    ix.names = ind + ["y"]
    out = pd.Series(index=ix,data=0)
    rslt =  im.groupby(ind)["y"].value_counts(normalize=True)
    out[rslt.index] = rslt.values
    return out
    

In [7]:
def get_weighting_matrix(
    data,
    empirical_moments,
    calc_moments,
    n_bootstrap_samples,
    n_draws_individuals,
    replace_missing_weights=None,
    return_covariance_matrix=False,
):
    """Compute a diagonal weighting matrix for estimation with MSM.
    Weights are the inverse bootstrap variances of the observed sample moments.
    Args:
    ------
    data (pandas.DataFrame)
        Dataframe containing individual observations. Must contain index named
        "Identifier" by which observations are sampled.
    empirical_moments (dict)
        Dictionary containing empirical moments in the form of pandas.DataFrame
        or pandas.Series.
    calc_moments (dict)
        Dictionary containing moment functions.
    n_bootstrap_samples (int)
        Number of samples that should be boostrapped.
    n_draws_individuals (int)
        Observations per bootstrap sample (individual ids).
    replace_missing_weights (None or float)
        Can be used to replace missing weights with a float value. If none, in
        cases where where weights are computed to be missing/infinite (i.e. if
        variances are 0), weights are set to zero.
    return_covariance_matrix : bool, default False
        Return full covariance matrix of bootstrapped moments.
    Returns:
    --------
    weighting_matrix (numpy.array)
        Diagonal weighting matrix with dimensions RxR where R denotes the
        number of moments.
    covariance_matrix (numpy.array)
        Covariance matrix of moments.
    """
    data = data.copy()
    np.random.seed(123)
    flat_empirical_moments = get_flat_moments(empirical_moments)
    index_base = data.index.get_level_values(0).unique()
    calc_moments = _harmonize_input(calc_moments)
    # Create bootstrapped moments.
    moments_sample = []
    for _ in range(n_bootstrap_samples):
        ids_boot = np.random.choice(index_base, n_draws_individuals, replace=False)
        moments_boot = {k: func(data.loc[ids_boot]) for k, func in calc_moments.items()}
        flat_moments_boot = get_flat_moments(moments_boot)
        flat_moments_boot = flat_moments_boot.reindex_like(flat_empirical_moments)
        # flat_moments_boot = flat_moments_boot.fillna(0)
        moments_sample.append(flat_moments_boot)

    # Compute variance for each moment and construct diagonal weighting matrix.
    moments_var = np.array(moments_sample).var(axis=0)

    # The variance of missing moments is nan. Unless a replacement variance is
    # specified, their inverse variance will be set to 0.
    diagonal = moments_var ** (-1)
    if replace_missing_weights is None:
        diagonal = np.nan_to_num(diagonal, nan=0, posinf=0, neginf=0)
    else:
        diagonal = np.nan_to_num(
            moments_var,
            nan=replace_missing_weights,
            posinf=replace_missing_weights,
            neginf=replace_missing_weights,
        )

    weighting_matrix = np.diag(diagonal)

    # Checks weighting matrix.
    if np.isnan(weighting_matrix).any() or np.isinf(weighting_matrix).any():
        raise ValueError("Weighting matrix contains NaNs or infinite values.")

    if return_covariance_matrix:
        covariance_matrix = np.cov(np.array(moments_sample).T, ddof=0)
        out = weighting_matrix, covariance_matrix
        assert np.allclose(
            moments_var, np.diag(covariance_matrix)
        ), "Variances in two outputs are not equal."
    else:
        out = weighting_matrix
    return out

In [8]:
def ordered_logit_msm(
    params,
    x,
    moment_func,
    moments_obs,
    cols,
    weighting=[],
    return_scalar=True
    
):
    """MSM criterion for ordered logit
    Maps params, independent data and moment obs in
    model fit.
    params: pd.DataFrame that contains parameters
    x: np.array containing independent variables for individuals
    moment_func: callable mapping data into moments
    moments_obs: observed moments
    cols: list of columns of data 
    
    """
    # parse the parameter vector into its quantities
    beta = params.loc["beta", "value"].to_numpy()
    cutoffs = params.loc["cutoff", "value"].to_numpy()

    # calculate deterministic part of utilities
    xb = x.dot(beta).reshape(len(x),1)

    # Simulate Result:
    upper_cutoffs = np.hstack([cutoffs, np.inf])
    lower_cutoffs = np.hstack([-np.inf, cutoffs])
    upper_cdf = stats.logistic.cdf(upper_cutoffs - xb)
    lower_cdf = stats.logistic.cdf(lower_cutoffs - xb)

    prob_cumulative = (upper_cdf - lower_cdf).cumsum(axis=1)
    draws = np.random.uniform(0,1,len(xb)).reshape(len(xb),1)
    labels = (draws < prob_cumulative).argmax(axis=1)
    
    moments_sim = moment_func(_build_data_df(x,labels,cols))
    
    dev = (moments_sim - moments_obs).values
    
    if len(weighting)==0:
        weighting = np.identity(len(moments_obs))
    
    if return_scalar:
        return dev @ weighting @ dev
    else:
        return dev

In [9]:
def ordered_logit_loglike(
    params,
    y,
    x
):
    """Likelihood function of an orderd logit model.
    taken from documentation 
    """
    # parse the parameter vector into its quantities
    beta = params.loc["beta", "value"].to_numpy()
    cutoffs = params.loc["cutoff", "value"].to_numpy()

    # calculate deterministic part of utilities
    xb = x.dot(beta)

    # evaluate likelihood
    upper_cutoffs = np.hstack([cutoffs, np.inf])[y]
    lower_cutoffs = np.hstack([-np.inf, cutoffs])[y]
    upper_cdf = stats.logistic.cdf(upper_cutoffs - xb)
    lower_cdf = stats.logistic.cdf(lower_cutoffs - xb)

    contributions = np.log(upper_cdf - lower_cdf)

    res = {"contributions": contributions, "value": contributions.sum()}

    return res


## Build Dataset


In [10]:
params = pd.DataFrame(pd.Series({
    ("beta","a"):-2,
    ("beta","b"):1,
    ("beta","c"):3,
    ("cutoff",0):2,
    ("cutoff",1):4,
}))
params.columns = ["value"]
params["lower_bound"] = - np.inf
params["upper_bound"] = np.inf

params.index = pd.MultiIndex.from_tuples(params.index)


In [11]:
params 

value  lower_bound  upper_bound
beta   a     -2         -inf          inf
       b      1         -inf          inf
       c      3         -inf          inf
cutoff 0      2         -inf          inf
       1      4         -inf          inf

In [12]:
data = simulate_dataset(50000, params)

In [13]:
#ind = ["a","b","c"] 
#im = data.copy()
#ranges = data.max(axis=0)
#ix = pd.MultiIndex.from_tuples(itertools.product(*(range(int(x + 1)) for x in ranges)))
#ix.names = ind + ["y"]
#out = pd.DataFrame(index=ix,data=0, columns=im.index)
#rslt =  im.groupby(ind)["y"]
#for i in out.columns:
    
#    out.loc[tuple(im.loc[i]),i] = 1
# Get Variance matrix here
#Salt = np.cov(np.array(out))
#W = np.diag(Salt)**(-1)

## Prepare data

In [14]:
# Data Set
#data = pd.read_pickle("~/OpenSourceEconomics/estimagic/docs/source/getting_started/ologit.pickle")
formula = "y ~ a + b + c"
start_params, y, x, constraints = ordered_logit_processing(formula, data)
n = x.shape[0]
n_agents_sim = 50000
cols = ["a","b","c","y"]

In [15]:
ind = list(start_params.loc["beta"].index.values)
moments_obs = _build_moments(data,ind)
moment_func = functools.partial(_build_moments,ind=ind)

Assume we are not allowed to keep the dependent information due to privacy concerns.
We are only allowed to extract a moments at a certain level of granularity.

In [16]:
weighting, S = get_weighting_matrix(
    data,
    moments_obs,
    moment_func,
    n_bootstrap_samples=1000,
    n_draws_individuals=100,
    replace_missing_weights=None,
    return_covariance_matrix=True,
)
S = S*100

/home/mo2561057/anaconda3/envs/ov_analysis/lib/python3.7/site-packages/ipykernel_launcher.py:60: RuntimeWarning: divide by zero encountered in reciprocal


In [17]:
# Now we pretend to leave our secure work space. Thus we have to delete y.
#del y
#del data

In [18]:
# Now we build the objective function
objective = functools.partial(
    ordered_logit_msm,
    x=x,
    moment_func=moment_func,
    moments_obs=moments_obs,
    weighting=weighting,
    cols=cols
)

In [19]:
# We perform one evaluation to make sure our setup works
objective(start_params)

4493.295891312718

In [20]:
# Optmize
rslt = minimize(
    criterion=objective,
    params=start_params,
    algorithm="scipy_powell",
    constraints=constraints,
    logging="ordered_logit.db",
)

In [21]:
params_msm = rslt["solution_params"] 

In [22]:
params_msm

group  lower_bound  upper_bound     value
type   name                                            
beta   a       beta         -inf          inf -1.963315
       b       beta         -inf          inf  0.946459
       c       beta         -inf          inf  2.885220
cutoff 0     cutoff         -inf          inf  1.989266
       1     cutoff         -inf          inf  3.857928

# Inference

In [23]:
def sandwich_cov(G, W, S, n):
    bread = np.linalg.inv(
        G.T @ W @ G
    )
    butter = G.T @ W @ S @ W @ G
    return bread @ butter @ bread / n

In [24]:
def get_msm_standart_errors(objective, theta_hat, S, weighting, n):
    # Get Hessian Matrix
    G = first_derivative(
    objective, 
    theta_hat, 
    method="central", 
    #key="moment_errors", 
    base_steps=0.3,
    return_func_value=True,
    n_cores=1,
    )[0].to_numpy()
    
    
    return sandwich_cov(G, W, S, n), G

In [25]:
objective = functools.partial(
    ordered_logit_msm,
    x=x,
    moment_func=moment_func,
    moments_obs=moments_obs,
    weighting=weighting,
    return_scalar=False,
    cols=cols
)

In [26]:
params

value  lower_bound  upper_bound
beta   a     -2         -inf          inf
       b      1         -inf          inf
       c      3         -inf          inf
cutoff 0      2         -inf          inf
       1      4         -inf          inf

In [27]:
cov, G = get_msm_standart_errors(objective, params_msm, S, weighting, n)

NameError: name 'W' is not defined

In [ ]:
np.sqrt(np.diag(cov))

In [ ]:
G.shape

In [ ]:
S

## Compare to likelihood errors


In [ ]:
# Data Set
formula = "y ~ a + b + c"
start_params, y, x, constraints = ordered_logit_processing(formula, data)
n = x.shape[0]
n_agents_sim = 50000
cols = ["a","b","c","y"]

In [ ]:
res = maximize(
    criterion=ordered_logit_loglike,
    params=start_params,
    algorithm="scipy_powell",
    constraints=constraints,
    criterion_kwargs={"y": y, "x": x},
    logging=False,
)

In [ ]:
params_lk = res["solution_params"]

In [ ]:
params_lk

In [ ]:
params_msm

## Se for likelihood

In [ ]:
from estimagic.decorators import numpy_interface

numpy_interface(ordered_logit_loglike, params=params_lk, constraints=constraints)

In [ ]:
inference, free_cov = do_likelihood_inference(
    loglike=ordered_logit_loglike,
    params=params_lk,
    loglike_kwargs={"x": x, "y": y},
    n_samples=50000,
    constraints=constraints,
)

inference.round(6)

In [ ]:
#inference